In [1]:
!pip3 install torch==1.7.1
!pip3 install tfrecord

  Using cached https://files.pythonhosted.org/packages/90/4f/acf48b3a18a8f9223c6616647f0a011a5713a985336088d7c76f3a211374/torch-1.7.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fe/ca/75fac5856ab5cfa51bbbcefa250182e50441074fdc3f803f6e76451fab43/dataclasses-0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/60/7a/e881b5abb54db0e6e671ab088d079c57ce54e8a01a3ca443f561ccadb37e/typing_extensions-3.7.4.3-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/45/b2/6c7545bb7a38754d63048c7696804a0d947328125d81bf12beaa692c3ae3/numpy-1.19.5-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fe/fd/247ef25f5ec5f9acecfbc98ca3c6aaf66716cf52509aca9a93583d410493/protobuf-3.14.0-cp36-cp36m-manylinux1_x86_64.whl
  Usin

In [2]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from tfrecord.torch.dataset import TFRecordDataset
import tensorflow as tf
import math
import numpy as np
from models import *

In [97]:
class Relational(nn.Module):
    def __init__(self,d_model, nhead, dff, nlayers, input_size, dropout = 0):
        super(Relational, self).__init__()
        from torch.nn import TransformerDecoderLayer, TransformerDecoder
        self.dense1 = nn.Linear(input_size, d_model)
        decoder_layers = TransformerDecoderLayer(d_model, nhead, dff, dropout)
        self.transformer_decoder = TransformerDecoder(decoder_layers, nlayers)
        #self.dense2 = nn.Linear(d_model, size_embedding)
        #self.gmm = 
    
    def get_last_time_step(self, tensor, stroke_lengths):
        
        embeddingd_lt = []
        
        for pos_embedding in range(tensor.shape[0]):
            embedding = tensor[pos_embedding, stroke_lengths[pos_embedding]-1,:]
            embeddingd_lt.append(embedding)
        
        embeddingd_lt = torch.vstack(embeddingd_lt) 
        
        return embeddingd_lt
    
    
    def forward(self, src, src_mask):
        #src = self.pos_encoder(src)
        output = self.dense1(src)
        #print(output[:,None,:].shape)
        output = self.transformer_decoder(output[:,None,:], output[:,None,:])
         
        #output = self.get_last_time_step(output, stroke_lengths)
        output = output[:,-1,:]
        
        #output = self.gmm(output)
        
        return output

In [19]:
path_data = "/data/jcabrera/didi_wo_text/training/"
name_file = "diagrams_wo_text_20200131-00000-of-00010"

In [20]:
tfrecord_path = path_data + name_file
index_path = None

In [21]:
description = {"key": "byte", "label_id":"byte",
               "ink":"float", "stroke_length":"int", "shape":"int", "num_strokes":"int", 
               "rdp_ink":"float", "rdp_stroke_length":"int", 
               "rdp_shape":"int", "rdp_num_strokes":"int",
               }

In [22]:
dataset = TFRecordDataset(tfrecord_path, index_path, description)
loader = torch.utils.data.DataLoader(dataset, batch_size=1)

In [23]:
dataiter = iter(loader)
. = dataiter.next()
print(data)

{'key': tensor([[ 48,  48,  51,  48,  55,  56,  57,  97,  57,  54,  97,  97, 101,  55,
         101,  97]], dtype=torch.uint8), 'label_id': tensor([[101,  50,  51,  56,  56,  97, 100,  56,  55,  98, 102,  55,  97,  52,
          98,  54,  49,  97,  51,  99,  52,  97,  55,  52, 102,  51,  49,  53,
          48, 101,  51, 100,  98,  97,  57,  56,  49, 100,  49,  97]],
       dtype=torch.uint8), 'ink': tensor([[0.1517, 0.5780, 0.0000,  ..., 0.0000, 0.0000, 0.0000]]), 'stroke_length': tensor([[ 32,  32,  25,  43, 128,  31,  33,  34,  29]], dtype=torch.int32), 'shape': tensor([[  9, 128,   4]], dtype=torch.int32), 'num_strokes': tensor([[9]], dtype=torch.int32), 'rdp_ink': tensor([[187.4000, 751.9100,   0.0000,  ...,   0.0000,   0.0000,   0.0000]]), 'rdp_stroke_length': tensor([[38, 36, 33, 21, 29, 19, 16, 13, 16]], dtype=torch.int32), 'rdp_shape': tensor([[ 9, 38,  4]], dtype=torch.int32), 'rdp_num_strokes': tensor([[9]], dtype=torch.int32)}


In [24]:
#d model cantidad de features: 64
#
xd = data["ink"].reshape((9,128,4))[:,:,0:3]
stroke_lengths = data["stroke_length"].numpy().reshape(-1)

In [25]:
stroke_lengths.reshape(-1)[0]

32

In [26]:
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")

In [27]:
encoder = Encoder(d_model=64, nhead=4, dff=128, nlayers=6, size_embedding=256, dropout = 0)

In [28]:
encoder = encoder.to(device)
xd = xd.to(device)

In [29]:
output = encoder(xd, stroke_lengths, None)

In [39]:
xd[:,0,:2]

tensor([[0.1517, 0.5780],
        [1.1419, 0.0811],
        [2.0582, 0.6355],
        [1.0001, 0.7735],
        [1.9281, 0.2507],
        [1.6074, 0.2034],
        [1.5922, 0.7573],
        [0.6347, 0.8220],
        [0.6258, 0.7169]], device='cuda:1')

In [30]:
xd.shape

torch.Size([9, 128, 3])

In [31]:
xd

tensor([[[1.5168e-01, 5.7804e-01, 0.0000e+00],
         [1.3512e-01, 5.8202e-01, 2.5000e+01],
         [1.2253e-01, 5.8489e-01, 4.5000e+01],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[1.1419e+00, 8.1085e-02, 9.9100e+02],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        [[2.0582e+00, 6.3554e-01, 1.9720e+03],
         [2.0203e+00, 6.3852e-01, 1.9940e+03],
         [1.9913e+00, 6.4745e-01, 2.0140e+03],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]],

        ...,

        [[1.5922e+00, 7.5728e-01, 8.1820e+03],
         [0.0000e+00, 0.0000e+00, 0.0000e+00]

In [32]:
output

tensor([[-0.1477,  0.0227, -0.9072,  ...,  0.2539,  0.1685,  0.7555],
        [-0.1477,  0.0228, -0.9072,  ...,  0.2539,  0.1684,  0.7556],
        [-0.3132, -0.0559, -0.8154,  ...,  0.0803,  0.1507,  0.8603],
        ...,
        [-0.2346, -0.0872, -0.6890,  ...,  0.4343,  0.2908,  0.7380],
        [-0.2346, -0.0873, -0.6890,  ...,  0.4344,  0.2908,  0.7379],
        [-0.2537, -0.0193, -0.8743,  ...,  0.1602,  0.1866,  0.8086]],
       device='cuda:1', grad_fn=<AddmmBackward>)

In [41]:
#initial pos x,y
init_pos = xd[:,0,:2]
init_pos.shape

torch.Size([9, 2])

In [33]:
#embedding
output.shape

torch.Size([9, 256])

In [58]:
pos_pred_input = torch.cat((output, init_pos), axis=1)

In [59]:
pos_pred_input.shape

torch.Size([9, 258])

In [102]:
#pos_pred_input = pos_pred_input[:,:,None]
pos_pred_input[:,None,:].shape

torch.Size([9, 1, 258])

In [103]:
pos_pred_input[:,None,:]

tensor([[[-0.1477,  0.0227, -0.9072,  ...,  0.7555,  0.1517,  0.5780]],

        [[-0.1477,  0.0228, -0.9072,  ...,  0.7556,  1.1419,  0.0811]],

        [[-0.3132, -0.0559, -0.8154,  ...,  0.8603,  2.0582,  0.6355]],

        ...,

        [[-0.2346, -0.0872, -0.6890,  ...,  0.7380,  1.5922,  0.7573]],

        [[-0.2346, -0.0873, -0.6890,  ...,  0.7379,  0.6347,  0.8220]],

        [[-0.2537, -0.0193, -0.8743,  ...,  0.8086,  0.6258,  0.7169]]],
       device='cuda:1', grad_fn=<SliceBackward>)

In [98]:
Position_prediction_model = Relational(d_model=64, nhead=4, dff=256, nlayers=6, input_size=256+2, dropout = 0)
Embedding_prediction_model = Relational(d_model=64, nhead=4, dff=256, nlayers=6, input_size=256+2+2, dropout = 0)

In [99]:
Position_prediction_model = Position_prediction_model.to(device)
Embedding_prediction_model = Embedding_prediction_model.to(device)

In [100]:
pos_pred = Position_prediction_model(pos_pred_input, None)

In [101]:
pos_pred.shape

torch.Size([9, 64])